# 事前準備

In [1]:
# 依存関係のインストール
%pip install -r ../requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.


In [14]:
# モジュールのインポート
import requests
import json
import os
from typing import Any, Callable, Set
from dotenv import load_dotenv
from pprint import pprint

from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet
from azure.ai.projects import AIProjectClient
from azure.cosmos import CosmosClient

In [15]:
# 環境変数の読み込み
load_dotenv(override=True)

URL = os.getenv("ACCOUNT_URI")
KEY = os.getenv("ACCOUNT_KEY")
PROJECT_CONNECTION_STRING = os.getenv("PROJECT_CONNECTION_STRING")
LOGIC_APPS = os.getenv("LOGIC_APPS")

AI Project クライアントを初期化します。

In [31]:
# AI Project Client の初期化
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=PROJECT_CONNECTION_STRING
)

ここでは、Azure AI Agent Service の Run で行われている処理を確認するためのユーティリティ関数を定義しています。

In [32]:
# ユーティリティ関数の定義
def print_run_steps(run_id: str, thread_id: str):
    """
    Run Step の詳細を表示する関数

    :param run_id: Run ID
    :rtype run_id: str
    :param thread: Thread ID
    :rtype run_id: str
    """
    ren_step_list = project_client.agents.list_run_steps(
        thread_id=thread_id, run_id = run_id
    )

    run_steps = reversed(ren_step_list["data"])

    for idx, step in enumerate(run_steps, start=1):
        print(f"\n=== Run Step {idx} ===")
        print(f"Type: {step['type']}")

        step_details = step['step_details']
        step_type = step_details.get('type')

        print("Details:")
        if step_type == 'tool_calls':
            for call in step_details.get('tool_calls', []):
                function = call.get('function', {})
                print(f"- Function Name: {function.get('name')}")
                print(f"- Arguments: {function.get('arguments')}")
        elif step_type == 'message_creation':
            message_id = step_details.get('message_creation').get('message_id')
            message = project_client.agents.get_message(
                thread_id=thread_id, message_id=message_id
            )
            print(message.content[0].text.value)
        else:
            pprint(step_details, indent=4, width=120)

# カスタム関数の定義

エージェントに持たせるカスタム関数を定義します。
まずは、先ほど作成した Cosmos DB からデータを取得する関数（`contract_lookup`）を作成します。

In [33]:
def contract_lookup(user_id: int) -> str:
    """
    ユーザーIDに基づいてユーザー情報を JSON 文字列で返します。ユーザー情報は変更しないでください。

    :param user_id (int): ユーザーのID
    :rtype: int

    :return: JSON 文字列のユーザー情報.
    :rtype: str
    """
    # Cosmos DB Client の初期化
    client = CosmosClient(url=URL, credential=KEY)
    database = client.get_database_client("cosmicworks")
    container = database.get_container_client("users")

    # ユーザー情報を取得するためのクエリ
    queryText = f'SELECT * FROM users u WHERE u.id = "{user_id}"'

    # Cosmos DB からユーザー情報を取得
    results = container.query_items(
        query=queryText,
        enable_cross_partition_query=True,
    )

    items = [item for item in results]
    output = items[0] if items else {}

    return json.dumps(output)


次に、Logic Apps を利用したメール送信関数（`send_email`）を定義します。

In [34]:
def send_email(customer: str, staff_email: str, inquiry: str) -> str:
    """
    お客様から担当者のメールアドレスにお問い合わせがあったことを通知

    :param customer (str): お客様の名前
    :rtype: str

    :param staff_email (str) : 担当者のメールアドレス
    :rtype: str

    :param inquiry (str) : 保険に関するお問い合わせ内容
    :rtype: str

    :return: JSON 文字列のユーザー情報.
    :rtype: str
    """
    headers = {"Content-Type": "application/json"}
    payload = {"customer": customer, "inquiry": inquiry, "staff_email": staff_email}

    endpoint_url = LOGIC_APPS

    try:
        response = requests.post(
            endpoint_url, headers=headers, data=json.dumps(payload)
        )
        response.raise_for_status()
        status = json.dumps({"status": "メールで通知が完了しました"})
        return status
    except requests.exceptions.RequestException as err:
        print(f"エラー: {err}")
        return json.dumps({"status": "メールで通知に失敗しました"})

最後に、`contract_lookup` 関数と `send_email` 関数を `ToolSet` オブジェクトに設定します。これにより、後でエージェントを作成する際に本ツールセットを指定して、これらのカスタム関数をエージェントのツールとして利用できるようになります。

In [35]:
# Toolset の作成＆関数の追加
user_functions: Set[Callable[..., Any]] = {contract_lookup, send_email}
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

# Azure AI Agent Service の実行

`create_agent` メソッドを用いてエージェント作成します。

In [36]:
# Agent の作成
agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="Contract Lookup Agent",
    instructions="""
あなたは、丁寧なアシスタントです。あなたは以下の業務を遂行します。
- 保険の契約をしているユーザーの情報について回答します
- お客様から担当者にお問い合わせメールを通知します

# 制約事項
- ユーザーからのメッセージは日本語で入力されます
- ユーザーからのメッセージから忠実に情報を抽出し、それに基づいて応答を生成します。
- ユーザーからのメッセージに勝手に情報を追加したり、不要な改行文字 \n を追加してはいけません

""", 
    toolset=toolset,
    headers={"x-ms-enable-preview": "true"},
)

print(f"Created Agent. AGENT_ID: {agent.id}")

Created Agent. AGENT_ID: asst_wntFXIjPuz54McKdd5sAqhMU


次に、スレッドを初期化し、ユーザーメッセージをスレッドに登録します。

In [37]:
# Thread の作成
thread = project_client.agents.create_thread()

print(f"Created Thread. THREAD_ID: {thread.id}")

Created Thread. THREAD_ID: thread_9FPs1Z1PPz4wyA1CavmAETiz


In [38]:
# メッセージの追加
user_message_01 = "ユーザー ID 1234 です。保険の加入状況を教えて。"

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=user_message_01,
)

print(f"Added Message. MESSAGE_ID: {message.id}")

Added Message. MESSAGE_ID: msg_6sjgkQvailrGRQ3Lqr1hfipw


次に、Run を実行し、エージェントの回答を生成します。

In [39]:
# Run 実行
run = project_client.agents.create_and_process_run(
    thread_id=thread.id, agent_id = agent.id
)
print(f"Run finished with status: {run.status}")

Run finished with status: RunStatus.COMPLETED


Run が正常に終了したら、スレッドからエージェントの回答を取得し出力します。

In [40]:
# 最新のレスポンスを取得
messages = project_client.agents.list_messages(thread_id=thread.id)
last_msg = messages.get_last_text_message_by_role("assistant")
response = last_msg.text.value if last_msg else None

if response is None:
    response = "No response found."
else:
    print(response)

ユーザー ID 1234 の加入情報は以下の通りです。

- お名前: 佐藤太郎
- 加入プラン: 安心保障プラン

何か他にご質問があればお知らせください。


エージェントがどのように回答を生成したのか、Run Steps を確認してみましょう。

In [41]:
# Run Step の詳細（Agent の動きを確認）
print_run_steps(run_id=run.id, thread_id=thread.id)



=== Run Step 1 ===
Type: tool_calls
Details:
- Function Name: contract_lookup
- Arguments: {"user_id":1234}

=== Run Step 2 ===
Type: message_creation
Details:
ユーザー ID 1234 の加入情報は以下の通りです。

- お名前: 佐藤太郎
- 加入プラン: 安心保障プラン

何か他にご質問があればお知らせください。


次に、メール送信を依頼します。同じスレッドにユーザーメッセージを書き込み、エージェントに Run を実行させることで続けて会話が可能です。

In [42]:
########################################
##  (２ターン目) 続けてメール送信を依頼  ##
########################################

# メッセージの追加
user_message_01 = "担当者に保険の見直しをお願いしたい。メールで通知して。"
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=user_message_01,
)

# Run 実行
run = project_client.agents.create_and_process_run(
    thread_id=thread.id, agent_id = agent.id
)

# 最新のレスポンスを取得
messages = project_client.agents.list_messages(thread_id=thread.id)
last_msg = messages.get_last_text_message_by_role("assistant")
response = last_msg.text.value if last_msg else None

if response is None:
    response = "No response found."
else:
    print(response)

担当者に「保険の見直しをお願いしたい」という内容でメールを送信し、通知を完了しました。担当者からの返信をお待ちください。何か他にサポートが必要であればお知らせください。


In [43]:
# Run Step の詳細（Agent の動きを確認）
print_run_steps(run_id=run.id, thread_id=thread.id)



=== Run Step 1 ===
Type: tool_calls
Details:
- Function Name: send_email
- Arguments: {"customer":"佐藤太郎","staff_email":"xxxxxxxx@microsoft.com","inquiry":"保険の見直しをお願いしたい。"}

=== Run Step 2 ===
Type: message_creation
Details:
担当者に「保険の見直しをお願いしたい」という内容でメールを送信し、通知を完了しました。担当者からの返信をお待ちください。何か他にサポートが必要であればお知らせください。


# （任意）プレイグラウンドで確認

作成したエージェントは、Azure AI Foundry 上のプレイグラウンドからも操作できるので確認してみましょう。

1. Azure Portal を開き、[リソースグループ] を展開。ここまで使用している該当のリソースグループを選択してください。
 ![image01-16](../images/image01-16.png)
 
2. リソースの一覧から演習0で作成した Azure AI Project を選択。
 ![image01-36](../images/image01-36.png)

4. その後、[Launch Studio] からAzure Ai Foundryを起動。
 ![image01-37](../images/image01-37.png)

 5. 起動後、画面左にある [エージェント] を選択。
 
 6. 作成した `Contract Lookup Agent` を選択、右の [プレイグラウンドで試す] を選択。
 ![image02-28](../images/image02-28.png) 

 7. プレイグラウンドで試す
 ![プレイグラウンドで確認](../images/image02-21.png)


# エージェントとスレッドの削除

最後に、エージェントとスレッドを削除します。

In [44]:
# Agent と Thread の削除
project_client.agents.delete_thread(thread_id=thread.id)
project_client.agents.delete_agent(agent_id=agent.id)
print("Agent と Thread を削除しました。")

Agent と Thread を削除しました。
